In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

In [3]:
import re
aspect_set = set()
with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/WOZ_dev_ans.txt") as f:
  for line in f:
    line = line.strip()
    line_lst = line.split("|")
    for aspect in line_lst[1:]:
      aspect_name = re.search(r"-(\w+)=(\w+)",aspect).group(1)
      aspect_set.add(aspect_name)


In [4]:
aspect_set.remove('name')
aspect_set.remove('food')

In [5]:
aspect_set

{'area', 'internet', 'parking', 'pricerange', 'stars', 'type'}

In [6]:
aspect_path = "/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/aspect_data/"

In [8]:
from collections import defaultdict
aspect_dict = defaultdict(set)
for aspect in list(aspect_set):
  with open(aspect_path + aspect + "_train.tsv", "w") as fout:
    with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/WOZ_train_ans.txt") as f_ans:
      with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/WOZ_train_utt.txt") as f_utt:
        for line_ans, line_utt in zip(f_ans, f_utt):
          line_ans = line_ans.strip()
          line_utt = line_utt.strip()
          line_lst = line_ans.split("|")
          label = "None"
          for aspect_item in line_lst[1:]:
            match = re.search(r"-(\w+)=(\w+)",aspect_item)
            if match and match.group(1) == aspect:
              label = match.group(2)
              aspect_dict[aspect].add(label)
          fout.write(line_utt + "\t" + label + "\n")


In [ ]:
for aspect in list(aspect_set):
  with open(aspect_path + aspect + "_dev.tsv", "w") as fout:
    with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/WOZ_dev_ans.txt") as f_ans:
      with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/WOZ_dev_utt.txt") as f_utt:
        for line_ans, line_utt in zip(f_ans, f_utt):
          line_ans = line_ans.strip()
          line_utt = line_utt.strip()
          line_lst = line_ans.split("|")
          label = "None"
          for aspect_item in line_lst[1:]:
            match = re.search(r"-(\w+)=(\w+)",aspect_item)
            if match and match.group(1) == aspect:
              label = match.group(2)
          fout.write(line_utt + "\t" + label + "\n")

In [9]:
aspect_dict

defaultdict(set,
            {'area': {'centre', 'dontcare', 'east', 'north', 'south', 'west'},
             'internet': {'dontcare', 'no', 'yes'},
             'parking': {'dontcare', 'no', 'yes'},
             'pricerange': {'cheap', 'dontcare', 'expensive', 'moderate'},
             'stars': {'0', '1', '2', '3', '4', '5', 'dontcare'},
             'type': {'dontcare', 'guesthouse', 'hotel'}})

In [10]:
aspect_lst = list(sorted(aspect_dict.keys()))
aspect_lst

['area', 'internet', 'parking', 'pricerange', 'stars', 'type']

In [26]:
with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/aspect_data/noncat_lab4_dev.txt") as f_cat:
  with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/WOZ_dev_utt.txt") as f_utt:
    with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/aspect_data/dev_stars.txt", "w") as fout:
      for line_cat, line_utt in zip(f_cat, f_utt):
        line_cat_lst = line_cat.strip().split("|")
        if "hotel-stars" in line_cat_lst:
          for char in line_utt:
            if char.isdigit():
              star = char
          fout.write(star + "\n")
        else:
          fout.write("None\n")

In [32]:
with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/aspect_data/noncat_lab4.txt") as f_cat:
  with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/WOZ_test_utt.txt") as f_utt:
    with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/aspect_data/pred_stars.txt", "w") as fout:
      for line_cat, line_utt in zip(f_cat, f_utt):
        line_cat_lst = line_cat.strip().split("|")
        if "hotel-stars" in line_cat_lst:
          for char in line_utt:
            if char.isdigit():
              star = char
          fout.write(star + "\n")
        else:
          fout.write("None\n")

In [ ]:
# with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/aspect_data/parking_train.tsv") as f:
#   for line in f:
#     line = line.strip()
#     line_lst = line.split("\t")
#     if line_lst[1] != "None":
#       print(line)

In [27]:
data_path = "/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/aspect_data/"
pred_lst_dev = []
with open(data_path + "noncat_lab4_dev.txt") as f:
  with open(data_path + "dev_area.txt") as f_area:
    with open(data_path + "dev_internet.txt") as f_internet:
      with open(data_path + "dev_parking.txt") as f_parking:
        with open(data_path + "dev_pricerange.txt") as f_pricerange:
          with open(data_path + "dev_stars.txt") as f_stars:
            with open(data_path + "dev_type.txt") as f_type:
              with open(data_path + "full_pred_dev.txt", "w") as fout:
                for line, line_area, line_internet, line_parking, line_pricerange, line_stars, line_type in zip(f, f_area, f_internet, f_parking, f_pricerange, f_stars, f_type):
                  line_lst = line.strip().split("|")
                  line_area = line_area.strip()
                  line_internet = line_internet.strip()
                  line_parking = line_parking.strip()
                  line_pricerange = line_pricerange.strip()
                  line_stars = line_stars.strip()
                  line_type = line_type.strip()
                  for i in range(1, len(line_lst)):
                    aspect = line_lst[i].split("-")[1]
                    for asp in aspect_lst:
                      # print(aspect)
                      if asp == aspect:
                        # print(asp)
                        # print(line_internet_lst)
                        # print(line_internet_lst[1])
                        # print(line_internet)
                        exec(f"label = line_{aspect}")
                        # print(label)
                        line_lst[i] = line_lst[i] + "=" + label
                  pred_lst_dev.append(("|".join(line_lst)))
                  fout.write("|".join(line_lst) + "\n")



In [ ]:
for i in range(len("find_hotel".split("|")[1:])):
  print(i)

In [52]:
import csv

fieldnames = ['ID', 'Expected']
data_path = "/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/aspect_data/"
with open(data_path + "noncat_lab4.txt") as f:
  with open(data_path + "pred_area.txt") as f_area:
    with open(data_path + "pred_internet.txt") as f_internet:
      with open(data_path + "pred_parking.txt") as f_parking:
        with open(data_path + "pred_pricerange.txt") as f_pricerange:
          with open(data_path + "pred_stars.txt") as f_stars:
            with open(data_path + "pred_type.txt") as f_type:
              with open(data_path + "full_pred.csv", "w") as fout:
                writer = csv.DictWriter(fout, fieldnames=fieldnames)
                index = 0
                writer.writeheader()
                for line, line_area, line_internet, line_parking, line_pricerange, line_stars, line_type in zip(f, f_area, f_internet, f_parking, f_pricerange, f_stars, f_type):
                  line_lst = line.strip().split("|")
                  line_area = line_area.strip()
                  line_internet = line_internet.strip()
                  line_parking = line_parking.strip()
                  line_pricerange = line_pricerange.strip()
                  line_stars = line_stars.strip()
                  line_type = line_type.strip()
                  for i in range(1, len(line_lst)):
                    aspect = line_lst[i].split("-")[1]
                    for asp in aspect_lst:
                      # print(aspect)
                      if asp == aspect:
                        # print(asp)
                        # print(line_internet_lst)
                        # print(line_internet_lst[1])
                        # print(line_internet)
                        exec(f"label = line_{aspect} if line_{aspect} != 'None' else 'dontcare'")
                        # print(label)
                        line_lst[i] = line_lst[i] + "=" + label
                  writer.writerow({'ID': index, 'Expected': "|".join(line_lst)})
                  index+=1


In [28]:

with open("/content/drive/MyDrive/Colab Notebooks/COLX_563_lab4/data/WOZ_dev_ans.txt") as f:
    dev_correct = [answer.strip() for answer in f.readlines()]  

In [29]:
accuracy_score(pred_lst_dev, dev_correct)

0.910411622276029